In [1]:
from mmaction.datasets import build_dataset
from mmaction.apis import train_model
from mmaction.models import build_model
from mmcv import Config, DictAction
import time
import os.path as osp
from mmaction.utils import collect_env, get_root_logger
from mmcv.runner import init_dist, set_random_seed

/home/kkallidromitis/anaconda3/envs/vfs/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_bgr=False)
train_pipeline = [
    dict(type='DecordInit'),
    dict(
        type='SampleFrames',
        clip_len=1,
        frame_interval=0,
        num_clips=2,
        out_of_bound_opt='loop'),
    dict(type='DecordDecode'),
    dict(
        type='RandomResizedCrop',
        area_range=(0.2, 1.),
        same_across_clip=False,
        same_on_clip=False),
    dict(type='Resize', scale=(224, 224), keep_ratio=False),
    dict(
        type='Flip',
        flip_ratio=0.5,
        same_across_clip=False,
        same_on_clip=False),
    # dict(
    #     type='ColorJitter',
    #     brightness=0.4,
    #     contrast=0.4,
    #     saturation=0.4,
    #     hue=0.1,
    #     p=0.8,
    #     same_across_clip=False,
    #     same_on_clip=False),
    # dict(
    #     type='RandomGrayScale',
    #     p=0.2,
    #     same_across_clip=False,
    #     same_on_clip=False),
    # dict(
    #     type='RandomGaussianBlur',
    #     p=0.5,
    #     same_across_clip=False,
    #     same_on_clip=False),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCTHW'),
    dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs', 'label'])
]

In [3]:
class arguments:
    pass
args=arguments()

args.config = 'configs/r50_sgd_cos_100e_r5_1xNx2_k400.py'
distributed = False
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
env_info_dict = collect_env()
env_info = '\n'.join([f'{k}: {v}' for k, v in env_info_dict.items()])
cfg = Config.fromfile(args.config)
cfg.gpu_ids = range(1)
cfg.seed = 0
cfg.work_dir = './work'

meta = dict()
meta['env_info'] = env_info
meta['seed'] = cfg.seed
meta['exp_name'] = osp.basename(args.config)


datasets = [build_dataset(cfg.data.train)]

In [4]:
datasets

In [5]:
model = build_model(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
train_model(model,datasets,cfg,distributed=distributed,validate=False,timestamp=timestamp,meta=meta)

2023-01-13 11:38:42,186 - mmaction - INFO - Start running, host: root@unagi, work_dir: /home/kkallidromitis/VFS/work
2023-01-13 11:38:42,187 - mmaction - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
after_train_epoch:


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/kkallidromitis/anaconda3/envs/vfs/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/kkallidromitis/anaconda3/envs/vfs/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/kkallidromitis/anaconda3/envs/vfs/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/kkallidromitis/VFS/mmaction/datasets/dataset_wrappers.py", line 26, in __getitem__
    return self.dataset[idx % self._ori_len]
  File "/home/kkallidromitis/VFS/mmaction/datasets/base.py", line 139, in __getitem__
    data = self.prepare_train_frames(idx)
  File "/home/kkallidromitis/VFS/mmaction/datasets/base.py", li

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x59b9b40] moov atom not found
[11:38:45] /github/workspace/src/video/video_reader.cc:83: ERROR opening: , Invalid data found when processing input
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7c371c0] moov atom not found
[11:38:46] /github/workspace/src/video/video_reader.cc:83: ERROR opening: , Invalid data found when processing input
